# AGNBoost Basic Usage Tutorial

This notebook demonstrates the basic workflow for using AGNBoost to predict AGN fractions from photometric data. We'll walk through:

1. Loading astronomical data with the Catalog class
2. Exploring the dataset structure and properties
3. Splitting data into training, validation, and test sets
4. Cleaning the data by removing rows with missing values
5. Loading a pre-trained AGN fraction model
6. Making predictions with uncertainty quantification
7. Evaluating model performance

Let's start by importing the necessary libraries and loading our data.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Set agnboost folder as root
import os

# Navigate to the repository root (parent directory of notebooks/)
os.chdir('..')

# Import necessary libraries
import numpy as np
import pandas as pd
from agnboost import dataset, model

# Set random seed for reproducibility
np.random.seed(123)

print("AGNBoost Basic Usage Tutorial")
print("=" * 40)

2025-05-31 20:10:51.393 | INFO     | agnboost.config:<module>:11 - PROJ_ROOT path is: /home/kurt/Documents/agnboost


AGNBoost Basic Usage Tutorial


## Loading the Data

We'll use the Catalog class to load our astronomical dataset. The `cigale_mock_small.csv` file contains is a small set of mock NIRCam+MIRI CIGALE galaxies for demonstration purposes.

In [3]:
# Load the astronomical data using the Catalog class
catalog = dataset.Catalog(path="data/cigale_mock_small.csv",summarize = False)


Current working directory: /home/kurt/Documents/agnboost
Looking for bands file at: /home/kurt/Documents/agnboost/allowed_bands.json
[INFO] Loaded bands file metadata: This file contains the allowed photometric bands for JWST
[INFO] Loaded 11 allowed bands from agnboost/allowed_bands.json
[INFO] Attempting to load file with delimiter: ','
[INFO] Successfully loaded data with 1000 rows.
[INFO] Found 11 valid band columns:
[INFO]   - jwst.nircam.F115W (F115W): 1.154 μm
[INFO]   - jwst.nircam.F150W (F150W): 1.501 μm
[INFO]   - jwst.nircam.F200W (F200W): 1.988 μm
[INFO]   - jwst.nircam.F277W (F277W): 2.776 μm
[INFO]   - jwst.nircam.F356W (F356W): 3.565 μm
[INFO]   - jwst.nircam.F410M (F410M): 4.083 μm
[INFO]   - jwst.nircam.F444W (F444W): 4.402 μm
[INFO]   - jwst.miri.F770W (F770W): 7.7 μm
[INFO]   - jwst.miri.F1000W (F1000W): 10.0 μm
[INFO]   - jwst.miri.F1500W (F1500W): 15.0 μm
[INFO]   - jwst.miri.F2100W (F2100W): 21.0 μm


In [11]:
catalog.allowed_bands

{'jwst.nircam.F115W': {'shorthand': 'F115W', 'wavelength': 1.154},
 'jwst.nircam.F150W': {'shorthand': 'F150W', 'wavelength': 1.501},
 'jwst.nircam.F200W': {'shorthand': 'F200W', 'wavelength': 1.988},
 'jwst.nircam.F277W': {'shorthand': 'F277W', 'wavelength': 2.776},
 'jwst.nircam.F356W': {'shorthand': 'F356W', 'wavelength': 3.565},
 'jwst.nircam.F410M': {'shorthand': 'F410M', 'wavelength': 4.083},
 'jwst.nircam.F444W': {'shorthand': 'F444W', 'wavelength': 4.402},
 'jwst.miri.F770W': {'shorthand': 'F770W', 'wavelength': 7.7},
 'jwst.miri.F1000W': {'shorthand': 'F1000W', 'wavelength': 10.0},
 'jwst.miri.F1500W': {'shorthand': 'F1500W', 'wavelength': 15.0},
 'jwst.miri.F2100W': {'shorthand': 'F2100W', 'wavelength': 21.0}}

## Exploring the Dataset

Let's examine the structure of our data to understand what photometric bands are available and get basic statistics about our dataset. The `print_data_summary()` method provides comprehensive information about:

- Dataset dimensions and memory usage
- Photometric band validation and metadata
- Column-by-column statistics including missing values
- Summary statistics for numerical columns

This information helps us understand data quality and identify any potential issues before modeling.

In [4]:
# Display comprehensive data summary
catalog.print_data_summary()

# Check which photometric bands were validated
valid_bands = catalog.get_valid_bands()
print(f"\nValid photometric bands found: {len(valid_bands)}")
for band_name, info in valid_bands.items():
    print(f"  {band_name}: {info['shorthand']} ({info['wavelength']} μm)")

# Check if our target variable exists
target_column = 'agn.fracAGN'
if target_column in catalog.get_data().columns:
    print(f"\nTarget variable '{target_column}' found in dataset")
    target_stats = catalog.get_data()[target_column].describe()
    print("Target variable statistics:")
    print(target_stats)
else:
    print(f"Warning: Target variable '{target_column}' not found in dataset")
    print("Available columns:", list(catalog.get_data().columns))


DATA SUMMARY: cigale_mock_small.csv
Dimensions: 1000 rows × 26 columns
Memory usage: 0.20 MB
--------------------------------------------------------------------------------
Valid Band Columns:
--------------------------------------------------------------------------------
Column Name                    Shorthand       Wavelength (μm)
--------------------------------------------------------------------------------
jwst.nircam.F115W              F115W           1.154          
jwst.nircam.F150W              F150W           1.501          
jwst.nircam.F200W              F200W           1.988          
jwst.nircam.F277W              F277W           2.776          
jwst.nircam.F356W              F356W           3.565          
jwst.nircam.F410M              F410M           4.083          
jwst.nircam.F444W              F444W           4.402          
jwst.miri.F770W                F770W           7.700          
jwst.miri.F1000W               F1000W          10.000         
jwst.miri.F15

## Creating Train/Test/Validation Splits

Before any modeling, we need to split our data into separate sets for training, validation, and testing. AGNBoost provides intelligent data splitting with optional stratification to ensure representative samples across all splits.

We'll use the default split ratios:
- 60% for training
- 20% for validation  
- 20% for testing

The random state ensures reproducible results. This step is not strictly necessary, as AGNBoost will internally perform the split if it has not explicitly been done.

In [5]:
# Create train/validation/test splitsget_train_val_test_sizes
catalog.split_data(test_size=0.2, val_size=0.2, random_state=42)

# Get split information
split_info = catalog.get_train_val_test_sizes()
print("Data split summary:")
print(f"  Total samples: {split_info['total']}")
print(f"  Training: {split_info['train']['size']} ({split_info['train']['percentage']:.1f}%)")
print(f"  Validation: {split_info['validation']['size']} ({split_info['validation']['percentage']:.1f}%)")
print(f"  Test: {split_info['test']['size']} ({split_info['test']['percentage']:.1f}%)")

Data split summary:
  Total samples: 1000
  Training: 600 (60.0%)
  Validation: 200 (20.0%)
  Test: 200 (20.0%)


## Cleaning the Data

Real astronomical datasets often contain missing values due to various observational limitations. Before training or making predictions, we will remove rows that have NaN values in critical columns.

The `drop_nan()` method removes rows with missing values in the validated photometric band columns, ensuring our model receives complete data for all features.

In [6]:
# Drop rows with NaN values in the validated columns
catalog.drop_nan(inplace=True)


[INFO] No rows with NaN values found in the specified columns.


There are no-nan rows to remove since the CIGALE mock data we loaded has none, but your real data might.

## Creating Features

AGNBoost automatically engineers features from photometric data, including colors and transformations. Let's create the feature dataframe that will be used for modeling.

By default, AGNBoost will create a features consisting of the photometric bands + derived colors + the squares of those derived colors

In [7]:
# Create features for modeling
catalog.create_feature_dataframe()

# Get information about created features
features = catalog.get_features()
print(f"Feature engineering complete:")
print(f"  Feature dataframe shape: {features.shape}")


[INFO] Created feature dataframe with 121 columns and 1000 rows.
[INFO] Created features are: ['jwst.nircam.F115W', 'jwst.nircam.F150W', 'jwst.nircam.F200W', 'jwst.nircam.F277W', 'jwst.nircam.F356W', 'jwst.nircam.F410M', 'jwst.nircam.F444W', 'jwst.miri.F770W', 'jwst.miri.F1000W', 'jwst.miri.F1500W', 'jwst.miri.F2100W', 'F2100W/F1500W', 'F2100W/F1000W', 'F2100W/F770W', 'F2100W/F444W', 'F2100W/F410M', 'F2100W/F356W', 'F2100W/F277W', 'F2100W/F200W', 'F2100W/F150W', 'F2100W/F115W', 'F1500W/F1000W', 'F1500W/F770W', 'F1500W/F444W', 'F1500W/F410M', 'F1500W/F356W', 'F1500W/F277W', 'F1500W/F200W', 'F1500W/F150W', 'F1500W/F115W', 'F1000W/F770W', 'F1000W/F444W', 'F1000W/F410M', 'F1000W/F356W', 'F1000W/F277W', 'F1000W/F200W', 'F1000W/F150W', 'F1000W/F115W', 'F770W/F444W', 'F770W/F410M', 'F770W/F356W', 'F770W/F277W', 'F770W/F200W', 'F770W/F150W', 'F770W/F115W', 'F444W/F410M', 'F444W/F356W', 'F444W/F277W', 'F444W/F200W', 'F444W/F150W', 'F444W/F115W', 'F410M/F356W', 'F410M/F277W', 'F410M/F200W', 'F41

## Loading the Pre-trained Model

AGNBoost comes with pre-trained models for common astronomical tasks. We'll load the model specifically trained for AGN fraction estimation (`agn.fracAGN`).

The `load_models()` method automatically:
- Checks for compatible pre-trained models
- Validates feature compatibility between the model and our data
- Loads model metadata including training parameters and performance metrics

In [8]:
# Initialize an AGNBoost model. The target variable is the name of the target variable column, and its value in the passed dictionary is the distribution used to model it.
agnboost_m = model.AGNBoost( feature_names = catalog.get_feature_names(),
                          target_variables = {'agn.fracAGN' : 'ZABeta'},
                         )

# Load pre-trained models. We will not pass a filename to load, and will simply the the most recent fracAGN model.
agnboost_m.load_model(model_name = 'fracAGN', overwrite = True)

if agnboost_m.models['agn.fracAGN'] is not None:
    print("✅ Pre-trained model loaded successfully!")
    
    # Display model information
    model_info = agnboost_m.model_info.get('agn.fracAGN', {})
    if model_info:
        print("\nModel information:")
        if 'training_timestamp' in model_info:
            print(f"  Trained: {model_info['training_timestamp']}")
        if 'best_score' in model_info:
            print(f"  Best validation score: {model_info['best_score']:.6f}")
        if 'features' in model_info:
            print(f"  Number of features: {len(model_info['features'])}")
else:
    print("❌ No pre-trained models found!")
    print("You may need to train a new model or check the models directory.")

2025-05-31 20:10:53,543 - AGNBoost.AGNBoost - WARNING - No file_name passed. Using the most recently modified one instead: 2025_05_22-PM06_59_58_agn.fracAGN_model.pkl.gz.


✅ Pre-trained model loaded successfully!

Model information:
  Best validation score: -649218.125000
  Number of features: 121


## Making Predictions

Now we'll use our loaded model to predict AGN fractions for the test set. AGNBoost seamlessly handles the conversion of our catalog data into the format required by the underlying XGBoost model.

The prediction process uses the engineered features (colors, log magnitudes, etc.) that were automatically created from our photometric band data.

In [9]:
# Make predictions on the test set
#agnboost_m.models['agn.fracAGN'].booster.set_param( {'device': 'cpu'})
preds = agnboost_m.predict( data = catalog, split_use = 'test', model_name = 'agn.fracAGN')

print(f"  Mean: {np.mean(preds):.6f}")
print(f"  Std: {np.std(preds):.6f}")
print(f"  Min: {np.min(preds):.6f}")
print(f"  Max: {np.max(preds):.6f}")


2025-05-31 20:10:55,651 - AGNBoost.AGNBoost - WARNING - Catalog object passsed. Taking the features and labels of the test set stored in the passed Catalog.


  Mean: 0.504962
  Std: 0.325251
  Min: 0.000308
  Max: 0.989859


## Quantifying Prediction Uncertainty

One of AGNBoost's key advantages is its ability to provide robust uncertainty estimates through XGBoostLSS distributional modeling. Rather than just point estimates, we get full uncertainty quantification for each prediction.

The `prediction_uncertainty()` method returns uncertainty estimates that account for both model uncertainty and the inherent variability in the data. This is crucial for astronomical applications where understanding prediction confidence is essential for scientific interpretation.

Since the loaded data is a CIGALE mock catalog with no photometric uncertainty, we will only estimate the model (aleatoric + epistemic) uncertainty for each source.

In [10]:
model_uncertainty = agnboost_m.prediction_uncertainty( uncertainty_type = 'model', model_name = 'agn.fracAGN', catalog = catalog)

print(f"✅ Uncertainty estimates generated")
print(f"Uncertainty statistics:")
print(f"  Mean uncertainty: {np.mean(model_uncertainty):.6f}")
print(f"  Std uncertainty: {np.std(model_uncertainty):.6f}")
print(f"  Min uncertainty: {np.min(model_uncertainty):.6f}")
print(f"  Max uncertainty: {np.max(model_uncertainty):.6f}")

2025-05-31 20:10:55,838 - AGNBoost.AGNBoost - WARNING - Catalog object passsed. Taking the features and labels of the None set stored in the passed Catalog.
Processing truncated model uncertainty:  38%|▍| 376/1000 [03:06<05:09,  2.02it/s


KeyboardInterrupt: 